In [1]:
cd /content/drive/MyDrive/20210611任务

/content/drive/MyDrive/20210611任务


In [2]:
!pip install transformers sentencepiece datasets rouge_score

     |████████████████████████████████| 3.3 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 57.1 MB/s 
     |████████████████████████████████| 298 kB 74.7 MB/s 
     |████████████████████████████████| 3.3 MB 40.1 MB/s 
     |████████████████████████████████| 596 kB 54.3 MB/s 
     |████████████████████████████████| 61 kB 649 kB/s 
     |████████████████████████████████| 895 kB 77.9 MB/s 
     |████████████████████████████████| 243 kB 56.1 MB/s 
     |████████████████████████████████| 1.1 MB 57.1 MB/s 
     |████████████████████████████████| 132 kB 62.3 MB/s 
     |████████████████████████████████| 271 kB 73.7 MB/s 
     |████████████████████████████████| 192 kB 77.9 MB/s 
     |████████████████████████████████| 160 kB 73.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


初步设想建立语义关系三元组（Q S A）
Q：问题
S：含义
A：答案文本


方案：
1.   利用Bert对医学样本建立词向量库
2.   使用GPT2增强Q样本
3.   Transformer对A进行S1提取
4.   Transformer对Q进行S2提取
5.   对比S1与S2的欧式距离，从而选出答案

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path 

import os

import torch
import torch.optim as optim

import random 
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from transformers import AdamW
from functools import partial

In [4]:
from sklearn.model_selection import train_test_split
f = open('./diseaseBERT/diseaseKnowledgeInfusionTraining/data/extractedQuestionAnswers_total_removeNoisy_maskedLM.txt','r')
total_text = f.read().split('\n')[:-1]

total_train,total_test,_,_ = train_test_split(total_text,[0 for line in total_text],test_size=0.2,random_state=66)

In [5]:
S_train_dic = {}
Q_train_dic = {}
A_train_dic = {}
for key, line in enumerate(total_train):
  SQA_temp = line.split('?\t')
  SQ_temp = SQA_temp[0].split('\t')
  S_train_dic[key] = SQ_temp[0] + ' : ' + SQ_temp[1]
  Q_train_dic[key] = SQ_temp[2]
  A_train_dic[key] = SQA_temp[1]

S_test_dic = {}
Q_test_dic = {}
A_test_dic = {}
for key, line in enumerate(total_test):
  SQA_temp = line.split('?\t')
  SQ_temp = SQA_temp[0].split('\t')
  S_test_dic[key] = SQ_temp[0] + ' : ' + SQ_temp[1]
  Q_test_dic[key] = SQ_temp[2]
  A_test_dic[key] = SQA_temp[1]

In [6]:
import pandas as pd

data = pd.DataFrame({'S':list(S_train_dic.values()),'Q':list(Q_train_dic.values()),'A':list(A_train_dic.values())})
data

,S,Q,A
0,gangrene : treatments,what are the treatments of gangrene,treatment varies based on the severity and typ...
1,multiple endocrine neoplasia type 1 : treatments,what are the treatments of multiple endocrine ...,the treatment of choice of parathyroid tumors ...
2,pancreatic pseudocyst : diagnosis,what are the diagnosis of pancreatic pseudocyst,diagnosis of pancreatic pseudocyst can be base...
3,oral submucous fibrosis : physiology,what are the physiology of oral submucous fibr...,"""exposure to areca nut (arecacatechu) containi..."
4,thrombophilia : diagnosis,what are the diagnosis of thrombophilia,tests for thrombophilia include complete blood...
...,...,...,...
11688,behçet's disease : treatments,what are the treatments of behçet's disease,current treatment is aimed at easing the sympt...
11689,tetany : general,what is tetany,tetany or tetanic seizure is a medical sign co...
11690,avulsion fracture : treatments,what are the treatments of avulsion fracture,"if the fracture is small, it is usually suffic..."
11691,lichen planus : treatments,what are the treatments of lichen planus,"there is no cure for lichen planus, when medic..."


# GPT2

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextGenerationPipeline

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [8]:
QStrain_data = []
QStest_data = []

for indexs in range(len(list(Q_train_dic.values()))):
  QStrain_data.append([Q_train_dic[indexs],S_train_dic[indexs]])
for indexs in range(len(list(Q_test_dic.values()))):
  QStest_data.append([Q_test_dic[indexs],S_test_dic[indexs]])

AStrain_data = []
AStest_data = []

for indexs in range(len(list(A_train_dic.values()))):
  AStrain_data.append([A_train_dic[indexs],S_train_dic[indexs]])
for indexs in range(len(list(A_test_dic.values()))):
  AStest_data.append([A_test_dic[indexs],S_test_dic[indexs]])

In [10]:
from tqdm.auto import tqdm
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(4)
all_data = []
for line in tqdm(QStrain_data):
  qTemp = line[0].split(" ")
  qTemp = generator(" ".join(qTemp[:len(qTemp)//3 * 2]), max_length=16, num_return_sequences=10)
  for dic in qTemp:
    all_data.append([dic['generated_text'].replace("\n",""), line[1]])
QStrain_data = all_data

In [ ]:
%%time
generator("what are the treatments", max_length=16, num_return_sequences=1)

In [25]:
%time p = generator("what are the treatments", max_length=16, num_return_sequences=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 1.64 s, sys: 8.93 ms, total: 1.65 s
Wall time: 1.65 s


In [44]:
from random import randint
new_list1 = []
new_list2 = []
num = list(Q_train_dic.keys())[-1]
for line in tqdm(list(Q_train_dic.values())):
  line = line.split(" ")
  try:
    index = randint(0,len(line) - 5)
    line = " ".join(line[index:index + 5])
  except:
    line = " ".join(line)
  new_list1.append(generator(line, max_length=16, num_return_sequences=2)[-1]['generated_text'])
  new_list2.append(generator(line, max_length=16, num_return_sequences=2)[-2]['generated_text'])

In [45]:
import numpy as np
total_list_Q = np.concatenate([list(Q_train_dic.values()),new_list1,new_list2],axis = -1)
total_list_A = np.concatenate([list(A_train_dic.values()),list(A_train_dic.values()),list(A_train_dic.values())],axis = -1)
total_list_S = np.concatenate([list(S_train_dic.values()),list(S_train_dic.values()),list(S_train_dic.values())],axis = -1)

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,Seq2SeqTrainingArguments
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Qtrain_encoding = tokenizer(
    total_list_Q,
    padding=True,
    truncation=True,
    max_length=16,
    return_tensors="pt"
)

Strain_label_encoding = tokenizer(
    total_list_S,
    padding=True,
    truncation=True,
    max_length=16,
    return_tensors="pt"
)

Qtest_encoding = tokenizer(
    list(Q_test_dic.values()),
    padding=True,
    truncation=True,
    max_length=16,
    return_tensors="pt"
)

Stest_label_encoding = tokenizer(
    list(S_test_dic.values()),
    padding=True,
    truncation=True,
    max_length=16,
    return_tensors="pt"
)


Atrain_encoding = tokenizer(
    total_list_A,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
)

Atest_encoding = tokenizer(
    list(A_test_dic.values()),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [14]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
 
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
 
    def __len__(self):
        return len(self.labels)
 
QStrain_dataset = IMDbDataset(Qtrain_encoding, Strain_label_encoding['input_ids'])
QStest_dataset = IMDbDataset(Qtest_encoding, Stest_label_encoding['input_ids'])

AStrain_dataset = IMDbDataset(Atrain_encoding, Strain_label_encoding['input_ids'])
AStest_dataset = IMDbDataset(Atest_encoding, Stest_label_encoding['input_ids'])

In [15]:
import nltk
import numpy as np
from datasets import load_metric
nltk.download('punkt')
metric = load_metric("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [46]:
#QS
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    "QS-summarization",
    evaluation_strategy = "epoch",
    num_train_epochs=12,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=QStrain_dataset,
    eval_dataset=QStest_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [64]:
features_in_hook = []
features_out_hook = []

def hook(module, fea_in, fea_out):
    features_in_hook.append(fea_in)
    features_out_hook.append(fea_out)
    return None

layer_name = 'decoder'
for (name, module) in model.named_modules():
    if name == layer_name:
        module.register_forward_hook(hook=hook)

In [ ]:
LHS_Q = []
for i in tqdm(range(len(Qtest_encoding["input_ids"]))):
  model.generate(Qtest_encoding["input_ids"][i:i+1].cuda(), do_sample=True, num_return_sequences=1, output_scores=True)
  LHS_Q.append(features_out_hook[0]['last_hidden_state'])
  features_out_hook = []

In [72]:
from transformers import pipeline
processor = pipeline('summarization', tokenizer=tokenizer, model=model.cpu())
S_Q = []
for line in total_list_Q:
  S_Q.append(processor(line)[-1]['summary_text'])

Your max_length is set to 200, but you input_length is only 5. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


'What the world has to offer a a bit of a boost to the euthanasia of the u.s.'

In [51]:
#AS
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    "AS-summarization",
    evaluation_strategy = "epoch",
    num_train_epochs=12,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=AStrain_dataset,
    eval_dataset=AStest_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [65]:
LHS_A = []
for i in tqdm(range(len(Qtest_encoding["input_ids"]))):
  model.generate(Qtest_encoding["input_ids"][i:i+1].cuda(), do_sample=True, num_return_sequences=1, output_scores=True)
  LHS_A.append(features_out_hook[0]['last_hidden_state'])
  features_out_hook = []

In [ ]:
from transformers import pipeline
processor = pipeline('summarization', tokenizer=tokenizer, model=model.cpu())
S_A = []
for line in total_list_A:
  S_A.append(processor(line)[-1]['summary_text'])

In [91]:
import numpy as np
def cosine_similarity(x,y):
  x = np.array(x).reshape([-1])
  y = np.array(y).reshape([-1])
  num = x.dot(y.T)
  denom = np.linalg.norm(x) * np.linalg.norm(y)
  return num / denom

In [ ]:
cosine_similarity(result[-1].cpu().data,result[-5].cpu().data)
LHS_Cos = []
for i, line in enumerate(LHS_Q):
  LHS_Cos.append(cosine_similarity(line.cpu().data,LHS_A[i].cpu().data))

In [55]:
pd.DataFrame({"Real_target":total_list_S,"LHS_Q":np.array(LHS_Q.cpu().data).reshape([-1]),"S_Q":S_Q,"LHS_A":np.array(LHS_A.cpu().data).reshape([-1]),"S_A",S_A}).to_csv("MiniMatching.csv",index = False)

In [ ]:
# pd.DataFrame({"Real_target":total_list_S,"LHS_Cos":LHS_Cos,"S_Q":S_Q,"S_A",S_A}).to_csv("MiniMatching.csv",index = False)